### Introduction
The Biogeographic Characterization Branch of CSASL partners with many organizations to produce the official inventory of protected open space in the United States, the Protected Areas Database of the United States (PAD-US). Working with federal, state, local, national and nongovernmental organizations, the PAD-US group assembles, checks and produces integrated information that describes public open space and other protected areas and delineates their boundaries. The resulting national inventory is an key resource for informing decisions about conservation, recreation or land use planning at different scales and across administrative boundaries.  

This notebook provides an overview of the inventory's assets and foundational data management practices. It also provides an entry point for exploring the interplay of managed lands, national conservation policy and resource management decisions. A goal of this notebook is to 

In [1]:
import requests
import pysb
import datetime
from IPython.display import display
from IPython.display import HTML

In [2]:
# To access private items in ScienceBase, we need to establish a connection using the pysb package. You will see summary output in this notebook from previous runs of the code when it is posted to GitHub, but we take care not to display particularly sensitive information from the in review items.

sb = pysb.SbSession()
username = input("Username: ")
sb.loginc(str(username))

Username: saulenbach@usgs.gov
········


In [3]:
# Set up some parameters for this notebook
_gc2BaseURL = "https://gc2.datadistillery.org/api/v1/sql/bcb"
_sbCatalogBaseURL = "https://www.sciencebase.gov/catalog/item/"

# ScienceBase item for PAD-US (currently v1.4)
_padusCollectionItem = "56bba648e4b08d617f657960"

### Summary Information
Retrieve select ScienceBase information to guide and support the analyses. 

In [4]:
# Get the collection item for PAD-US v1.4 for the  summary
padusCollection = sb.get_item(_padusCollectionItem,{'fields':'title,body,purpose,contacts'})

# Display a few summary fields for the collection
_shortReport = "<h3>"+padusCollection["title"]+"</h4>"
_shortReport = _shortReport+"<h4>Abstract</h4><p>"+padusCollection["body"]+"</p>"
_shortReport = _shortReport+"<h4>Purpose</h4><p>"+padusCollection["purpose"]+"</p>"
_shortReport = _shortReport+"<h4>Contacts</h4>"
for contact in padusCollection["contacts"]:
    _shortReport = _shortReport+"<div>"
    _shortReport = _shortReport+contact["name"]+" ("+contact["type"]+")"
    _shortReport = _shortReport+"</div>"
    
HTML(_shortReport)

In [5]:
# Report on the current number of PAD-US items in ScienceBase
padusIDs = sb.get_child_ids(_padusCollectionItem)
_shortReport = "The number of PAD-US v1.4 items in the ScienceBase collection is currently <strong>"+str(len(padusIDs))+"</strong></div>"

# Get a short list of PAD-US items with a search and show a summary
for padusID in padusIDs:
    padusItem = sb.get_item(padusID)
    # Display a few summary fields for the collection
    pid = _sbCatalogBaseURL+padusID
    _shortReport = _shortReport+"<ul>"+padusItem["title"]+" ("+str(padusID)+")"+"</ul>"
    _
HTML(_shortReport)